In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install import-ipynb
import import_ipynb

In [ ]:
%cd drive
%cd 'My Drive'
%cd 'RL'
%cd 'Homework'
%cd 'Homework 4'


In [19]:
'''Needed Libraries and Environment'''
import Environment
import Modified_Environment
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from collections import defaultdict
import sys
import itertools
import matplotlib
import pandas as pd
import pickle

'''Constants Defined'''
STATE = 0
ACTION = 1
REWARD = 2
MAX_T = 10000
TOTAL_EPISODES = 2000
gamma = 0.9
ALPHA = 0.2
MIN = 0.1
DECAY_FACTOR = 0.99

In [22]:
class nStepExpectedSarsa():
  def __init__(self, env, gamma, n = 2):
      self.env = env
      self.gamma = gamma
      self.n = n
      self.actions = env.action_space.n
      self.nstep_expsarsa_episode_scores = []
      self.Q = defaultdict(lambda: np.zeros(env.action_space.n))

  def epsilon_greedy_policy(self, state, epsilon):
      epsilon_policy = np.ones(self.actions, dtype=float) * epsilon / self.actions
      best_action = np.argmax(self.Q[state])
      epsilon_policy[best_action] += (1.0 - epsilon)
      return epsilon_policy

  def reach_island(self):
      #epsilon = 1 #<-- uncomment if you want to use decaying epsilon
      #epsilon = 0.5 #<-- uncomment if you want to use decaying epsilon (bonus part)
      #epsilon = 0.3 #<-- uncomment if you want to use constant epsilon
      for episode in range(1, TOTAL_EPISODES + 1):
          episode_score = 0
          if episode % 2 == 0:
                print("\rEpisode {}/{}.".format(episode, TOTAL_EPISODES), end="")
                sys.stdout.flush()
                #epsilon = max(epsilon * DECAY_FACTOR, MIN)  #<-- uncomment if you want to use decaying epsilon
          stored_rewards, stored_states, stored_actions  = {}, {}, {}        
          T, t, tau = sys.maxsize, -1, 0
          state = env.reset()
          probs = self.epsilon_greedy_policy(state, epsilon)
          action = np.random.choice(np.arange(len(probs)), p = probs)

          stored_states[0], stored_actions[0]  = state, action
          
          while tau < (T - 1):
              t += 1
              if t < T:
                  next_state, reward, done, _ = env.step(action)
                  state = next_state
                  stored_states[(t + 1) % (self.n + 1)],  stored_rewards[(t + 1) % (self.n + 1)] = state, reward 
                  episode_score += reward   
                  
                  if done or t > MAX_T: T = t + 1
                  else:
                      probs = self.epsilon_greedy_policy(state, epsilon)
                      action = np.random.choice(np.arange(len(probs)), p = probs)
                      stored_actions[(t + 1) % (self.n + 1)] = action
              
              tau = t - self.n + 1
              information = [stored_states, stored_actions, stored_rewards]
              self.expected_sarsa_returns(information, tau, T, epsilon)
          self.nstep_expsarsa_episode_scores.append(episode_score)

  def expected_sarsa_returns(self, info, tau, T, epsilon):
      if tau >= 0:
          G = np.sum([(self.gamma**(i-tau-1)) * info[REWARD][i % (self.n + 1)] for i  in range(tau + 1, min(tau + self.n, T) + 1)])
          
          if tau + self.n < T:
              exp_sarsa_update = np.sum([self.epsilon_greedy_policy(info[STATE][(tau + self.n) % (self.n + 1)], epsilon)[a] * self.Q[info[STATE][(tau + self.n) % (self.n + 1)]][a] for a in range(self.actions)])
              G += (self.gamma ** self.n) * exp_sarsa_update
          
          s_tau, a_tau = info[STATE][tau % (self.n + 1)], info[ACTION][tau % (self.n + 1)]
          self.Q[s_tau][a_tau] += ALPHA * (G - self.Q[s_tau][a_tau])
  

In [31]:
#env =  Environment.GridworldEnv() #<--uncomment for normal environment
#env =  Modified_Environment.GridworldEnv() # <--uncomment for modified environment (bonus part)
agent = nStepExpectedSarsa(env, gamma)
agent.reach_island()

Episode 2000/2000.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(TOTAL_EPISODES), agent.nstep_expsarsa_episode_scores, color = '#633974',label = '$\epsilon$ = from $0.5$ to $0.1$')
plt.xlabel('episodes ->')
plt.ylabel('epsiode score ->')
plt.title('2-Step Expected SARSA')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
window_size = 50
smoothed_score = pd.Series(agent.nstep_expsarsa_episode_scores).rolling(window_size, min_periods=window_size).mean()
plt.plot(smoothed_score, color = '#633974', label = '$\epsilon$ = from $0.5$ to $0.1$')
plt.annotate(smoothed_score.iloc[-1], xy=(2000, smoothed_score.iloc[-1]), xytext=(2000, smoothed_score.iloc[-1]), color = '#633974',) # <--uncomment for modified environment (bonus part)
plt.xlabel("epsiode ->")
plt.ylabel("epsiode score (smoothed) -> ")
plt.title('2-Step Expected SARSA')
plt.legend()
plt.show()

In [34]:
#i wrote my reward data into a .txt file to use it later.

# with open('nstep_expsarsa_bonus_scores', 'wb') as fp:
#     pickle.dump(agent.nstep_expsarsa_episode_scores, fp)